Data Wrangling Notebook for Sykes Bone Data
<br />
Neeka Sewnath
<br />
nsewnath@ufl.edu

TODO: 
- Do we want to remove what's in ()? Is verbatimScientificName a thing now?
- Where should bone ID be perserved?
- What element does 1 fit in with? Ontology has 1 but no element attached. 
- Check additional requried columns
- Check measurementUnit
- Check exporting name

In [1]:
import pandas as pd
import numpy as np
import re
import uuid 

Silencing warnings that are unnecessary

In [2]:
try:
    import warnings
    warnings.filterwarnings('ignore')
except:
    pass

Import Sykes Bone Data

In [3]:
data = pd.read_csv("./../Original_Data/sykes_bone_data.csv")

Add individualID and populate with UUID

In [ ]:
data = data.assign(individualID = '')
data['individualID'] = [uuid.uuid4().hex for _ in range(len(data.index))]

Set samplingProtocol and measurementMethod

In [ ]:
data = data.assign(samplingProtocol = data["reference"], measurementMethod = data["reference"])

Adding additional required GEOME columns

In [ ]:
data = data.assign(country = "", 
                   yearCollected = "", 
                   locality = "", 
                   basisOfRecord = "")

Renaming columns

In [ ]:
data = data.rename(columns = {'':''})

In [ ]:
# some more pre mapping stuff here #

Renaming measurementType values

In [ ]:
# Replace names of terms avaliable in GEOME
# NOTE: Make sure mapping file is up to date before reprocessing (git pull from FuTRES Repo)

# Read mapping file 
mapping_file = pd.read_csv("./../Mapping Files/ontology_codeBook.csv")

# Create subset of those within FOVT or OBA
map_subset = mapping_file[(mapping_file["Status"] == "in FOVT") | (mapping_file["Status"] == "in OBA") ]

# Create a subset of Joaquin data
sykes_subset = map_subset[map_subset["name"] == "Sykes"]

# Isolating necessary columns
sykes_subset = sykes_subset[["label", "term"]]

# Create dictionary of terms
map_dict = sykes_subset.set_index('label').to_dict()['term']

# Map the new terms onto the old terms in the dataframe 
data["measurementType"] = (pd.Series(data["measurementType"])).map(map_dict)

In [ ]:
# some more post mapping stuff #

Create materialSampleID which is a UUID for each measurement. Populate eventID with materialSampleID

In [ ]:
data['materialSampleID'] = [uuid.uuid4().hex for _ in range(len(data.index))]
data = data.assign(eventID = data["materialSampleID"])

Create diagnosticID

In [ ]:
data['diagnosticID'] = data['diagnosticID'] = np.arange(len(data))

Adding measurementUnit column

In [ ]:
data['measurementUnit'] = ""

Write file to csv

In [ ]:
data.to_csv('../Mapped_Data/FuTRES_Sykes_Dama.csv')